In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_pickle("../grab-ai-safety-data/df_full.pickle")

In [4]:
np.round(df.describe())

,bookingid,accuracy,bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,speed,label
count,1.453895e+07,14538950.0,14538950.0,14538950.0,14538950.0,14538950.0,14538950.0,14538950.0,14538950.0,1.453895e+07,14538950.0,14538950.0
mean,8.185184e+11,12.0,169.0,0.0,4.0,1.0,-0.0,0.0,-0.0,3.687000e+03,9.0,0.0
std,4.953506e+11,87.0,107.0,1.0,8.0,3.0,0.0,0.0,0.0,1.414407e+06,8.0,0.0
min,0.000000e+00,1.0,0.0,-78.0,-73.0,-76.0,-48.0,-75.0,-37.0,0.000000e+00,-2.0,0.0
25%,3.779571e+11,4.0,78.0,-1.0,-2.0,-1.0,-0.0,-0.0,-0.0,2.410000e+02,1.0,0.0
50%,8.074539e+11,4.0,169.0,0.0,9.0,1.0,-0.0,0.0,-0.0,5.200000e+02,8.0,0.0
75%,1.254130e+12,8.0,263.0,1.0,10.0,3.0,0.0,0.0,0.0,8.630000e+02,15.0,1.0
max,1.709397e+12,6070.0,360.0,67.0,75.0,78.0,40.0,80.0,66.0,1.495797e+09,148.0,1.0


In [ ]:
mask = df["bookingid"] != df["bookingid"].shift(1)

df["change_in_second"] = df["second"].diff().fillna(0)
df["change_in_bearing"] = np.abs(df["bearing"].diff().fillna(0))
df["change_in_speed"] = np.abs(df["speed"].diff().fillna(0))
df["change_in_acx"] = df["acceleration_x"].diff().fillna(0)
df["change_in_acy"] = df["acceleration_y"].diff().fillna(0)
df["change_in_acz"] = df["acceleration_z"].diff().fillna(0)
df["change_in_gyx"] = df["gyro_x"].diff().fillna(0)
df["change_in_gyy"] = df["gyro_y"].diff().fillna(0)
df["change_in_gyz"] = df["gyro_z"].diff().fillna(0)

df.loc[ mask, [
    "change_in_second", 
    "change_in_bearing",
    "change_in_speed",
    "change_in_acx",
    "change_in_acy",
    "change_in_acz",
    "change_in_gyx",
    "change_in_gyy",
    "change_in_gyz"
]] = 0

## Duration of trip
- This one is just obvious. Clear difference in average durations between safe and dangerous trips

In [ ]:
df.groupby("bookingid").agg(
    {
        "second" : np.max,
        "label" : np.max
    }
).groupby("label").agg("mean")

## Distance of trip
- There is a weird entry with a change in time at the last entry
- Looks like there are 5 trips with ridiculous trip durations

In [ ]:
df_dist = df.copy()
df_dist["distance_covered"] = df_dist["change_in_second"] * df_dist["speed"]
df_dist.groupby("bookingid").agg(
    {
        "distance_covered" : np.sum,
        "label" : np.max
    }
).groupby("label").mean()

In [ ]:
df_dist.loc[df_dist["bookingid"] == 1503238553722, :][list(df.columns)]["second"].max()

In [ ]:
duration_check = df_dist.groupby("bookingid").agg(
    {
    "second" : np.max,
    "label" : np.max
    }
)

In [ ]:
duration_check.loc[duration_check["label"] == 0, :] \
              .groupby("second") \
              .count().sort_values("second", ascending=False).reset_index()["second"][1]

## Acceleration
- Let's try euclidean first to see if that works
    - Mean of bookings, mean of label: no significant difference ~1% higher only
    - Max of the bookings, mean of label: ~12% higher for dangerous trips

In [ ]:
df_acc = df.copy()
df_acc["acceleration"] = np.sqrt(
    (df_acc["acceleration_x"] ** 2) + (df_acc["acceleration_y"] ** 2) + (df_acc["acceleration_z"] ** 2)
)
test = df_acc.groupby("bookingid").agg(
    {
        "acceleration" : [np.max, np.mean],
        "label" : [np.max]
    }
)
test.columns = test.columns.get_level_values(0)
test.groupby("label").agg("mean")

## Gyro
- Same technique with gyro to see
    - Max of bookings, mean of label: ~47% higher for dangerous trips
    - Mean of bookings, mean of label: ~36% higher for dangerous trips

In [ ]:
df_gyro = df.copy()
df_gyro["gyro"] = np.sqrt(
    (df_gyro["gyro_x"] ** 2) + (df_gyro["gyro_y"] ** 2) + (df_gyro["gyro_z"] ** 2)
)
test = df_gyro.groupby("bookingid").agg(
    {
        "gyro" : [np.max, np.mean],
        "label" : [np.max]
    }
)
test.columns = test.columns.get_level_values(0)
test.groupby("label").agg("mean")

## Speed
- More variations for speed
    - Max of bookings, mean of label: 5% higher for dangerous trips
    - Mean of bookings, mean of label: 10% lower for dangerous trips

In [ ]:
df_spd = df.copy()
test = df_spd.groupby("bookingid").agg(
    {
        "speed" : [np.max, np.mean],
        "label" : [np.max]
    }
)
test.columns = test.columns.get_level_values(0)
test.groupby("label").agg("mean")

## Bearing
- Change in bearing?